In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_excel(
    r"source_data/Тестовое_задание_Data_аналитик_ЦО_2025_.xlsx",
    sheet_name="Данные для задачи 2"
)

df.head()

,Дата,Код товара,Количество продано,Розничная цена,Цена с учётом скидки
0,2024-06-01,3226300004,9,271,271.0
1,2024-06-02,3226300004,15,271,271.0
2,2024-06-03,3226300004,12,271,216.8
3,2024-06-04,3226300004,17,271,216.8
4,2024-06-05,3226300004,13,271,216.8


In [3]:
# Переименовываем для удобства
data = df.rename(columns={
    "Дата": "date",
    "Код товара": "sku",
    "Розничная цена": "retail_price",
    "Цена с учётом скидки": "price",
    "Количество продано": "qty"
})

# Рассчитываем фактическую скидку
data["discount"] = (data["retail_price"] - data["price"]) / data["retail_price"]

data.head()

,date,sku,qty,retail_price,price,discount
0,2024-06-01,3226300004,9,271,271.0,0.0
1,2024-06-02,3226300004,15,271,271.0,0.0
2,2024-06-03,3226300004,12,271,216.8,0.2
3,2024-06-04,3226300004,17,271,216.8,0.2
4,2024-06-05,3226300004,13,271,216.8,0.2


In [4]:
def calculate_elasticity(group):
   """
   Рассчитывает эластичность для одного товара по формуле:
   \nln(qty) = a + b * ln(price)
   \n где b — коэффициент эластичности спроса по цене
   """
   X = np.log(group[["price"]].values)
   y = np.log(group["qty"].values)

   # Используем линейную регрессию из sklearn для оценки коэффициентов
   model = LinearRegression()
   model.fit(X, y)
   # Получаем коэффициент эластичности
   elasticity = model.coef_[0]

   return pd.Series({"elasticity": elasticity})

# Группируем по товару и рассчитываем эластичность
elasticity = data.groupby("sku").apply(calculate_elasticity).reset_index()

elasticity.head(10)

/tmp/ipykernel_298684/1994289091.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  elasticity = data.groupby("sku").apply(calculate_elasticity).reset_index()


,sku,elasticity
0,3226300004,-2.583489
1,3234800007,-2.936029
2,14011400047,-2.714853
3,14012900005,-3.087862
4,15110900016,-2.547794
5,15111300006,-3.213509
6,15111300017,-3.702950
7,19000000363,-2.723476
8,19000013047,-2.829271
9,19760336592,-2.655730


In [5]:
# Средние метрики по каждому товару
current_metrics = data.groupby("sku").agg({
    "price": "mean",           # средняя цена после скидки
    "discount": "mean",        # средняя скидка
    "qty": "mean",             # средние продажи в день
    "retail_price": "median"   # розничная цена (берём медиану)
}).reset_index()

current_metrics.columns = ["sku", "current_price", "current_discount", "current_qty", "retail_price"]

current_metrics.head()

,sku,current_price,current_discount,current_qty,retail_price
0,3226300004,260.232787,0.055738,13.032787,271.0
1,3234800007,330.685246,0.044262,12.672131,346.0
2,14011400047,1525.786885,0.063934,13.721311,1630.0
3,14012900005,1154.832787,0.055738,13.008197,1223.0
4,15110900016,1415.960656,0.042623,11.877049,1479.0


In [6]:
# Объединяем текущие метрики и эластичность
result = current_metrics.merge(elasticity, on="sku")

def calc_target_price(row):
    """
    Рассчитывает целевую цену для одного товара по формуле:
    \ntarget_price = current_price * (2 ** (1 / b))
    \n где b — коэффициент эластичности спроса по цене
    """
    b = row["elasticity"]
    return row["current_price"] * (2 ** (1 / b))


# Рассчитываем целевую цену
result["target_price"] = result.apply(calc_target_price, axis=1)
# Рассчитываем целевую скидку
result["target_discount"] = (result["retail_price"] - result["target_price"]) / result["retail_price"]
# Переводим в проценты
result["target_discount"] = result["target_discount"] * 100
result.head()

,sku,current_price,current_discount,current_qty,retail_price,elasticity,target_price,target_discount
0,3226300004,260.232787,0.055738,13.032787,271.0,-2.583489,198.994607,26.570256
1,3234800007,330.685246,0.044262,12.672131,346.0,-2.936029,261.147073,24.523967
2,14011400047,1525.786885,0.063934,13.721311,1630.0,-2.714853,1181.983108,27.485699
3,14012900005,1154.832787,0.055738,13.008197,1223.0,-3.087862,922.637190,24.559510
4,15110900016,1415.960656,0.042623,11.877049,1479.0,-2.547794,1078.693290,27.066039


In [7]:
final = result[[
    "sku",
    "retail_price",
    "target_price",
    "target_discount"
]].copy()

final = final.round({
    "retail_price": 0,
    "target_price": 0,
    "target_discount": 0
})

final.head(10)

,sku,retail_price,target_price,target_discount
0,3226300004,271.0,199.0,27.0
1,3234800007,346.0,261.0,25.0
2,14011400047,1630.0,1182.0,27.0
3,14012900005,1223.0,923.0,25.0
4,15110900016,1479.0,1079.0,27.0
5,15111300006,8057.0,6238.0,23.0
6,15111300017,19396.0,15531.0,20.0
7,19000000363,2062.0,1528.0,26.0
8,19000013047,3117.0,2288.0,27.0
9,19760336592,2816.0,2077.0,26.0


In [8]:
# Сохраняем результат
final.to_excel("output_data/task2_result.xlsx", index=False)